In [1]:
import pandas as pd
import numpy as np
import pickle
import spacy
import scispacy
from collections import Counter
import pathlib
from spacytextblob.spacytextblob import SpacyTextBlob
from scispacy.abbreviation import AbbreviationDetector
from scispacy.linking import EntityLinker
from negspacy.negation import Negex
from negspacy.termsets import termset
ts = termset("en_clinical")

# Intantiate the spacy program specifying the model you want to use
nlp = spacy.load("en_core_sci_lg")

# Add the abbreviation pipe to the spacy pipeline.
nlp.add_pipe("abbreviation_detector")
# Add the hpo entity linker pipe to the spacy pipeline.
nlp.add_pipe("scispacy_linker", config={"resolve_abbreviations": True,
                                        "linker_name": "umls",
                                        'threshold':0.85,
                                        'max_entities_per_mention':1
                                       })
# add the sentiment analysis step
nlp.add_pipe('spacytextblob')
# add the negation detection
nlp.add_pipe("negex")
ts.add_patterns({
            "pseudo_negations": [],
            "termination": [],
            "preceding_negations": ["was no", "were no", "were never", "showed no", "a negative"],
            "following_negations": ["was negative", "remained negative", "was undetectable"],
        })

# nlp.remove_pipe("scispacy_linker")

/home/jcampbell/miniconda3/envs/gopher/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.20.3 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/home/jcampbell/miniconda3/envs/gopher/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.20.3 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [2]:
paed_path = '/home/jcampbell/paed_covid_case_reports/picu_control_set/output/retrieved_df/retrieved_df2.p'
df = pickle.load(open(paed_path, 'rb'))   
condition = [type(text) == str for text in df['content_text']]
df=pd.DataFrame(df.loc[condition,:])
print(len(df))

1427


In [58]:
# # abbreviations
# abrvs = {'long_forms':[],'spans':[]}
# for abrv in doc._.abbreviations:
#     abrvs['long_forms'].append(abrv._.long_form)
#     abrvs['spans'].append((abrv.start,abrv.end))
    
# Counter(abrvs['long_forms'])

In [57]:


# # text = 'I had a really horrible day. It was the worst day ever! But every now and then I have a really good day that makes me happy.'
# doc = nlp(df['content_text'][1])
# for sent in doc.sents:
#     polarity = sent._.polarity
#     if polarity < -0.5:
#         print(sent)
#         print(sent._.assessments)
# # ent = doc.ents[10]
# # print(ent)
# # # doc._.polarity      # Polarity: -0.125
# # # doc._.subjectivity  # Sujectivity: 0.9
# # # doc._.assessments   # Assessments: [(['really', 'horrible'], -1.0, 1.0, None), (['worst', '!'], -1.0, 1.0, None), (['really', 'good'], 0.7, 0.6000000000000001, None), (['happy'], 0.8, 1.0, None)]
# # ent._.subjectivity

In [54]:
# from negspacy.termsets import termset
# ts = termset("en_clinical")
# print(ts.get_patterns())
# ts.add_patterns({
#             "pseudo_negations": [],
#             "termination": [],
#             "preceding_negations": ["was no", "were no", "were never", "showed no", "a negative"],
#             "following_negations": ["was negative", "remained negative", "was undetectable"],
#         })

In [51]:

# # from negspacy.negation import Negex

# doc = nlp(df['content_text'][2])
# # nlp = spacy.load("en_core_web_sm")
# # nlp.add_pipe("negex")

# # doc = nlp("She does not like Steve Jobs but likes Apple products.")
# for e in doc.ents:
#     if e._.negex == True:
#         print(doc[e.start:e.end])
#         print(e.sent)
#         print('\n')
# #     print(e.text, e._.negex)

In [3]:
hpo_ents = {}
for counter, doc in enumerate(nlp.pipe(list(df['content_text']), batch_size =100)):
    # every doc will get a list entities that map to HPO
    # each entity will have a dictionary with a series of values
    hpos = []
    # iterate through each entity in the doc
    for entity in doc.ents:
        # try mapping to HPO
        linker = nlp.get_pipe("scispacy_linker")
        if entity._.kb_ents:
            # if there is a "match/mention" you get a cui
            mention = entity._.kb_ents[0]
            if mention:
                # then we get the cui details (mainly the name)
                hpo = linker.kb.cui_to_entity[mention[0]]
                if hpo is not None:
                    # the result is a sring that needs a bit of parsing
                    hpo = str(hpo).split('\n')[0].split(', ')
                    # we then store attributes for each entity starting with HPO CUI
                    hpos.append({'cui':hpo[0].replace('CUI: ',''),
                                 # then we add the primary long form name
                                 'name':hpo[1].replace('Name: ',''),
                                 # then we add the start and end point of the entity (span)
                                 'span':(entity.start, entity.end),
                                 # then the score given for the confidence of the match
                                 'hpo_thresh':mention[1],
                                 # negation of the entity bool
                                 'negation':entity._.negex,
                                 # the sentiment for the sentence the entity is from
                                 'polarity': entity.sent._.polarity,
                                 # if the sentence is mostly opinion or fact
                                 'subjectivity':entity.sent._.subjectivity
                                })
    # now we add the ent_d list to our df_d               
    hpo_ents.update({df.index[counter]:hpos})
    if counter % 10 == 0:
        print(f'{counter} of {len(df)} complete')
# for each hpo we want the cui, name and threshold




/home/jcampbell/miniconda3/envs/gopher/lib/python3.8/site-packages/scispacy/candidate_generation.py:284: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  extended_neighbors[empty_vectors_boolean_flags] = numpy.array(neighbors)[:-1]
/home/jcampbell/miniconda3/envs/gopher/lib/python3.8/site-packages/scispacy/candidate_generation.py:285: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  extended_distances[empty_vectors_boolean_flags] = numpy.array(distances)[:-1]


0 of 1427 complete
10 of 1427 complete
20 of 1427 complete
30 of 1427 complete
40 of 1427 complete
50 of 1427 complete
60 of 1427 complete
70 of 1427 complete
80 of 1427 complete
90 of 1427 complete
100 of 1427 complete
110 of 1427 complete
120 of 1427 complete
130 of 1427 complete
140 of 1427 complete
150 of 1427 complete
160 of 1427 complete
170 of 1427 complete
180 of 1427 complete
190 of 1427 complete
200 of 1427 complete
210 of 1427 complete
220 of 1427 complete
230 of 1427 complete
240 of 1427 complete
250 of 1427 complete
260 of 1427 complete
270 of 1427 complete
280 of 1427 complete
290 of 1427 complete
300 of 1427 complete
310 of 1427 complete
320 of 1427 complete
330 of 1427 complete
340 of 1427 complete
350 of 1427 complete
360 of 1427 complete
370 of 1427 complete
380 of 1427 complete
390 of 1427 complete
400 of 1427 complete
410 of 1427 complete
420 of 1427 complete
430 of 1427 complete
440 of 1427 complete
450 of 1427 complete
460 of 1427 complete
470 of 1427 complete
480

In [68]:
def val_to_df(ent_list):
    cols = list(ent_list[0].keys())
    # holding lists
    cuis = []
    names = []
    spans =[]
    hpo_thresh = []
    negation = []
    polarity = []
    subjectivity = []
    for val in ent_list:
        cuis.append(val['cui'])
        names.append(val['name'])
        spans.append(val['span'])
        hpo_thresh.append(val['hpo_thresh'])
        negation.append(val['negation'])
        polarity.append(val['polarity'])
        subjectivity.append(val['subjectivity'])
    df = pd.DataFrame(data = {'cui':cuis,
                              'name':names,
                              'span':spans,
                              'hpo_thresh':hpo_thresh,
                              'negation':negation,
                              'polarity':polarity,
                              'subjectivity':subjectivity})
    return df

In [74]:
ent_df = val_to_df(hpo_ents[df.index[1]])
len(ent_df)

101

In [75]:
Counter(ent_df['name']).most_common(10)

[('Increased number of platelets in blood', 10),
 ('Abnormality of the cerebrospinal fluid', 10),
 ('Lymphocytopenia', 8),
 ('Severe', 7),
 ('Fever', 7),
 ('Low platelet count', 6),
 ('Low neutrophil count', 5),
 ('Irritability', 4),
 ('Bone infection', 4),
 ('Thrombocytopenia', 4)]

In [90]:
for d in hpos:
    if d['name'] == 'Severe':
        print(doc[d['span'][0]].sent)
        print('\n')

Diabetic ketoacidosis at the onset of T1DM was more frequent, and a higher percentage of these patients presented with a severe form.


Several studies reported a significant increase in the frequency of DKA present at the diagnosis of T1DM, with a higher percentage of the severe form of this metabolic complication [,,,,,].


The degree of severity of DKA was classified according to 2018 International Society for Paediatric and Adolescent Diabetes (ISPAD) guidelines: mild DKA (pH < ), moderate DKA (pH < ), severe DKA (pH < ).


In the pandemic group, a higher percent of DKA cases developed the severe form compared to the pre-pandemic group ( vs. , OR = , CI 95% = –, p = ). (, ).


Among the eight patients, seven presented DKA which was mild in three cases, moderate in three, and severe in one patient.


Moreover, a higher percentage of children presented a severe form of this metabolic complication.




In [4]:
pickle.dump(hpo_ents, open('./umls_ents.p', 'wb'))

In [79]:
create ent_sents

SyntaxError: invalid syntax (<ipython-input-79-bf186ad93750>, line 1)